# *De Novo* Molecule Reasoning with PROTON

This notebook enables reasoning over de novo molecules without retraining the PROTON model by:

1. Using **Uni-Mol2**, a pre-trained structure-aware molecular encoder, to generate embeddings from SMILES strings.
2. Training a small **adapter network** to map Uni-Mol2 representations into PROTON's 512-dimensional embedding space.
3. Using the projected embeddings with PROTON's decoder for **drug-disease predictions**.

The adapter is trained using the 8,160 existing drugs in NeuroKG, teaching it to recreate PROTON's learned drug embeddings from their chemical structures.

## Setup and Dependencies

Install molecular encoding dependencies (Uni-Mol2 and RDKit).

In [ ]:
# Install dependencies for molecular encoding
# Uncomment and run if not already installed
# !uv add "unimol_tools<0.1.5" "lifelines<0.29.0" rdkit-pypi

In [1]:
import logging
import os
import sys
import time
from pathlib import Path


def _find_repo_root(start: Path) -> Path:
    """Find PROTON-GEM root by walking up to `pyproject.toml`."""
    start = start.resolve()
    for parent in (start, *start.parents):
        if (parent / "pyproject.toml").exists():
            return parent
    return start


# Ensure all relative paths are resolved from the PROTON-GEM project root.
PROJECT_ROOT = _find_repo_root(Path.cwd())
os.chdir(PROJECT_ROOT)
if str(PROJECT_ROOT) not in sys.path:
    sys.path.insert(0, str(PROJECT_ROOT))

import matplotlib.font_manager as fm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pytorch_lightning as pl
import requests
import seaborn as sns
import torch
import torch.nn as nn
import torch.nn.functional as F
import wandb
import copy
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
from tqdm import tqdm

from src.config import conf
from src.constants import TORCH_DEVICE
from src.dataloaders import load_graph
from src.models import HGT

# Load environment variables from secrets.env
try:
    from dotenv import load_dotenv
except Exception:  # pragma: no cover
    load_dotenv = None

if load_dotenv is not None:
    load_dotenv(conf.paths.secrets_path)

if any("Arial" in f.name for f in fm.fontManager.ttflist):
    plt.rcParams["font.family"] = "Arial"

_logger = logging.getLogger(__name__)

Define output directories.

In [2]:
# Output directories
DRUGS_DIR = Path("data/neurokg/drugs")
DRUGS_DIR.mkdir(parents=True, exist_ok=True)

OUTPUT_DIR = Path("data/notebooks/molecular_analysis")
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

# Cache file paths
SMILES_CACHE_PATH = DRUGS_DIR / "drug_smiles.csv"
UNIMOL_CACHE_PATH = DRUGS_DIR / "unimol_embeddings.pt"
ADAPTER_PATH = conf.paths.checkpoint.base_dir / "molecular_adapter.pt"

## 2. Load PROTON Model and Drug Data

Read CSVs of nodes and edges.

In [3]:
nodes = pd.read_csv(conf.paths.kg.nodes_path, dtype={"node_index": int}, low_memory=False)
edges = pd.read_csv(
    conf.paths.kg.edges_path, dtype={"edge_index": int, "x_index": int, "y_index": int}, low_memory=False
)

_logger.info(f"Number of nodes: {len(nodes)}")
_logger.info(f"Number of edges: {len(edges) / 2:.0f}")

[02/02/26 02:34:42] INFO     Number of nodes: 147020                                                ]8;id=650696;file:///tmp/ipykernel_3590678/2575215186.py\2575215186.py]8;;\:]8;id=166744;file:///tmp/ipykernel_3590678/2575215186.py#6\6]8;;\

                    INFO     Number of edges: 7366745                                               ]8;id=674483;file:///tmp/ipykernel_3590678/2575215186.py\2575215186.py]8;;\:]8;id=381291;file:///tmp/ipykernel_3590678/2575215186.py#7\7]8;;\

Load knowledge graph and PROTON model.

In [4]:
pl.seed_everything(conf.seed, workers=True)
kg = load_graph(nodes, edges)
pretrain_model = HGT.load_from_checkpoint(
    checkpoint_path=str(conf.paths.checkpoint.checkpoint_path),
    kg=kg,
    strict=False,
)
pretrain_model.eval()
pretrain_model = pretrain_model.to(TORCH_DEVICE)

[rank: 0] Seed set to 42


Load pre-computed PROTON embeddings.

In [5]:
embeddings = torch.load(conf.paths.checkpoint.embeddings_path)
_logger.info(f"Embeddings shape: {embeddings.shape}")

EMBEDDING_DIM = embeddings.shape[1]
_logger.info(f"PROTON embedding dimension: {EMBEDDING_DIM}")

[02/02/26 02:35:05] INFO     Embeddings shape: torch.Size([147020, 512])                            ]8;id=234053;file:///tmp/ipykernel_3590678/1825762581.py\1825762581.py]8;;\:]8;id=146316;file:///tmp/ipykernel_3590678/1825762581.py#2\2]8;;\

                    INFO     PROTON embedding dimension: 512                                        ]8;id=571858;file:///tmp/ipykernel_3590678/1825762581.py\1825762581.py]8;;\:]8;id=91161;file:///tmp/ipykernel_3590678/1825762581.py#5\5]8;;\

Extract drug nodes and their embeddings.

In [6]:
# Filter to drug nodes
drug_nodes = nodes[nodes["node_type"] == "drug"].copy().reset_index(drop=True)
_logger.info(f"Number of drug nodes: {len(drug_nodes)}")

# Extract drug embeddings from PROTON
drug_indices = drug_nodes["node_index"].values
drug_embeddings = embeddings[drug_indices]
_logger.info(f"Drug embeddings shape: {drug_embeddings.shape}")

[02/02/26 02:35:06] INFO     Number of drug nodes: 8160                                             ]8;id=243962;file:///tmp/ipykernel_3590678/2311303571.py\2311303571.py]8;;\:]8;id=529903;file:///tmp/ipykernel_3590678/2311303571.py#3\3]8;;\

                    INFO     Drug embeddings shape: torch.Size([8160, 512])                         ]8;id=681453;file:///tmp/ipykernel_3590678/2311303571.py\2311303571.py]8;;\:]8;id=735392;file:///tmp/ipykernel_3590678/2311303571.py#8\8]8;;\

## 3. SMILES Data Collection

Fetch SMILES structures for drugs in NeuroKG from PubChem. Results are cached to avoid repeated API calls.

In [7]:
def get_smiles_from_pubchem(drugbank_id: str, timeout: int = 10) -> str | None:
    """Fetch SMILES from PubChem given a DrugBank ID.

    Args:
        drugbank_id: DrugBank identifier (e.g., 'DB00001')
        timeout: Request timeout in seconds

    Returns:
        Isomeric SMILES string if found, None otherwise
    """
    url = f"https://pubchem.ncbi.nlm.nih.gov/rest/pug/compound/name/{drugbank_id}/property/IsomericSMILES/JSON"
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36"
    }
    try:
        response = requests.get(url, headers=headers, timeout=timeout)
        response.raise_for_status()
        data = response.json()
        return data["PropertyTable"]["Properties"][0]["IsomericSMILES"]
    except Exception:
        return None


def fetch_drug_smiles(
    drug_nodes: pd.DataFrame,
    cache_path: Path,
    rate_limit: float = 0.1,
 ) -> pd.DataFrame:
    """Fetch SMILES for all drugs, using cache if available.

    Args:
        drug_nodes: DataFrame with drug node information
        cache_path: Path to cache CSV file
        rate_limit: Seconds to wait between API calls

    Returns:
        DataFrame with SMILES column added
    """
    # Check for cached data
    if cache_path.exists():
        _logger.info(f"Loading cached SMILES from {cache_path}")
        cached = pd.read_csv(cache_path, dtype={"node_index": int})
        return cached

    # Fetch SMILES from PubChem
    _logger.info("Fetching SMILES from PubChem API...")
    smiles_list = []
    for drug_id in tqdm(drug_nodes["node_id"], desc="Fetching SMILES"):
        smiles = get_smiles_from_pubchem(drug_id)
        smiles_list.append(smiles)
        time.sleep(rate_limit)  # Rate limiting

    # Add SMILES to dataframe
    result = drug_nodes[["node_index", "node_id", "node_name", "node_type"]].copy()
    result["SMILES"] = smiles_list

    # Save cache
    result.to_csv(cache_path, index=False)
    _logger.info(f"Saved SMILES cache to {cache_path}")

    return result

Fetch or load SMILES data.

In [8]:
drug_smiles_df = fetch_drug_smiles(drug_nodes, SMILES_CACHE_PATH)

# Report coverage
total_drugs = len(drug_smiles_df)
valid_smiles = drug_smiles_df["SMILES"].notna().sum()
coverage = 100 * valid_smiles / total_drugs

_logger.info(f"Retrieved SMILES for {valid_smiles}/{total_drugs} drugs ({coverage:.1f}%)")

[02/02/26 02:38:40] INFO     Loading cached SMILES from data/neurokg/drugs/drug_smiles.csv         ]8;id=848749;file:///tmp/ipykernel_3590678/2448122167.py\2448122167.py]8;;\:]8;id=911527;file:///tmp/ipykernel_3590678/2448122167.py#41\41]8;;\

                    INFO     Retrieved SMILES for 6944/8160 drugs (85.1%)                           ]8;id=356778;file:///tmp/ipykernel_3590678/2452081164.py\2452081164.py]8;;\:]8;id=291369;file:///tmp/ipykernel_3590678/2452081164.py#8\8]8;;\

Filter to drugs with valid SMILES.

In [9]:
# Filter to valid SMILES
valid_drugs_df = drug_smiles_df[drug_smiles_df["SMILES"].notna()].copy().reset_index(drop=True)
_logger.info(f"Drugs with valid SMILES: {len(valid_drugs_df)}")

# Get corresponding PROTON embeddings for valid drugs
valid_drug_indices = valid_drugs_df["node_index"].values
valid_drug_embeddings = embeddings[valid_drug_indices]
_logger.info(f"Valid drug embeddings shape: {valid_drug_embeddings.shape}")

[02/02/26 02:38:45] INFO     Drugs with valid SMILES: 6944                                          ]8;id=398382;file:///tmp/ipykernel_3590678/2574706329.py\2574706329.py]8;;\:]8;id=101414;file:///tmp/ipykernel_3590678/2574706329.py#3\3]8;;\

                    INFO     Valid drug embeddings shape: torch.Size([6944, 512])                   ]8;id=846335;file:///tmp/ipykernel_3590678/2574706329.py\2574706329.py]8;;\:]8;id=45561;file:///tmp/ipykernel_3590678/2574706329.py#8\8]8;;\

## Uni-Mol2 Molecular Encoding

Use the pre-trained Uni-Mol2 model to encode SMILES strings into 512-dimensional molecular representations.

In [10]:
def encode_molecules_unimol(
    smiles_list: list[str],
    cache_path: Path | None = None,
    batch_size: int = 32,
) -> tuple[torch.Tensor, list[int]]:
    """Encode SMILES strings using Uni-Mol2.

    Args:
        smiles_list: List of SMILES strings
        cache_path: Optional path to cache embeddings
        batch_size: Batch size for encoding

    Returns:
        tuple[Tensor, list[int]]:
            - Tensor of molecular embeddings [N_valid, 512]
            - List of indices of successfully encoded molecules
    """
    # Check cache
    if cache_path is not None and cache_path.exists():
        _logger.info(f"Loading cached Uni-Mol2 embeddings from {cache_path}")
        data = torch.load(cache_path)
        # Handle new cache format (dict) vs old format (Tensor)
        if isinstance(data, dict) and "embeddings" in data and "valid_indices" in data:
            return data["embeddings"], data["valid_indices"]
        elif isinstance(data, torch.Tensor):
            # Fallback for old cache: assume all molecules were valid if cache exists
            # Verify length match to be safe
            if len(data) == len(smiles_list):
                 return data, list(range(len(data)))
            else:
                _logger.warning("Cached embeddings length mismatch. Recomputing.")

    # Import Uni-Mol2
    try:
        from unimol_tools import UniMolRepr
    except ImportError as e:
        raise ImportError(
            "Uni-Mol2 not installed. Run: pip install unimol_tools"
        ) from e

    # Initialize Uni-Mol2 model
    _logger.info("Loading Uni-Mol2 model...")
    clf = UniMolRepr(data_type="molecule", remove_hs=False)

    # Encode in batches
    _logger.info(f"Encoding {len(smiles_list)} molecules...")
    all_embeddings = []
    valid_indices = []

    for i in tqdm(range(0, len(smiles_list), batch_size), desc="Encoding molecules"):
        batch = smiles_list[i : i + batch_size]
        batch_indices = range(i, i + len(batch))
        
        try:
            # UniMolRepr returns dict with 'cls_repr' key containing [batch, 512] embeddings
            reprs = clf.get_repr(batch, return_atomic_reprs=False)
            # Extract CLS token representation (molecule-level)
            batch_emb = torch.tensor(reprs["cls_repr"], dtype=torch.float32)
            all_embeddings.append(batch_emb)
            valid_indices.extend(batch_indices)
        except ValueError as e:
            _logger.warning(f"Batch encoding failed: {e}. Retrying individually.")
            # Retry individually to isolate the bad SMILES
            for j, smile in enumerate(batch):
                try:
                    reprs = clf.get_repr([smile], return_atomic_reprs=False)
                    emb = torch.tensor(reprs["cls_repr"], dtype=torch.float32)
                    all_embeddings.append(emb)
                    valid_indices.append(batch_indices[j])
                except ValueError:
                    _logger.warning(f"Skipping illegal SMILES at index {batch_indices[j]}: {smile}")
                    continue

    if not all_embeddings:
        raise ValueError("No valid molecules found!")

    embeddings = torch.cat(all_embeddings, dim=0)
    _logger.info(f"Uni-Mol2 embeddings shape: {embeddings.shape}")

    # Save cache
    if cache_path is not None:
        torch.save({"embeddings": embeddings, "valid_indices": valid_indices}, cache_path)
        _logger.info(f"Saved Uni-Mol2 embeddings to {cache_path}")

    return embeddings, valid_indices

Encode drugs with Uni-Mol2.

In [ ]:
# Get SMILES list for valid drugs
smiles_list = valid_drugs_df["SMILES"].tolist()

# Encode with Uni-Mol2
unimol_embeddings, valid_unimol_indices = encode_molecules_unimol(smiles_list, cache_path=UNIMOL_CACHE_PATH)
_logger.info(f"Uni-Mol2 embeddings shape: {unimol_embeddings.shape}")

# Filter data if needed to match valid encodings
if len(valid_unimol_indices) < len(smiles_list):
    n_dropped = len(smiles_list) - len(valid_unimol_indices)
    _logger.warning(f"Dropping {n_dropped} molecules due to encoding failures.")
    
    # Filter targets (embeddings)
    valid_drug_embeddings = valid_drug_embeddings[valid_unimol_indices]
    
    # Filter metadata
    valid_drugs_df = valid_drugs_df.iloc[valid_unimol_indices].reset_index(drop=True)
    
    _logger.info(f"Filtered valid drug embeddings shape: {valid_drug_embeddings.shape}")

[02/02/26 02:38:55] INFO     Enabling RDKit 2022.09.5 jupyter extensions                             ]8;id=82627;file:///n/holylfs06/LABS/mzitnik_lab/Users/rzhu/PROTON-GEM/.venv/lib64/python3.11/site-packages/rdkit/__init__.py\__init__.py]8;;\:]8;id=578856;file:///n/holylfs06/LABS/mzitnik_lab/Users/rzhu/PROTON-GEM/.venv/lib64/python3.11/site-packages/rdkit/__init__.py#22\22]8;;\

[02/02/26 02:38:57] INFO     Loading Uni-Mol2 model...                                              ]8;id=379201;file:///tmp/ipykernel_3590678/883461153.py\883461153.py]8;;\:]8;id=605397;file:///tmp/ipykernel_3590678/883461153.py#30\30]8;;\

2026-02-02 02:38:57 | unimol_tools/weights/weighthub.py | 32 | INFO | Uni-Mol Tools | Weights will be downloaded to default directory: /n/holylfs06/LABS/mzitnik_lab/Users/rzhu/PROTON-GEM/.venv/lib64/python3.11/site-packages/unimol_tools/weights


                    INFO     Weights will be downloaded to default directory:                       ]8;id=810620;file:///n/holylfs06/LABS/mzitnik_lab/Users/rzhu/PROTON-GEM/.venv/lib64/python3.11/site-packages/unimol_tools/weights/weighthub.py\weighthub.py]8;;\:]8;id=303445;file:///n/holylfs06/LABS/mzitnik_lab/Users/rzhu/PROTON-GEM/.venv/lib64/python3.11/site-packages/unimol_tools/weights/weighthub.py#32\32]8;;\
                             /n/holylfs06/LABS/mzitnik_lab/Users/rzhu/PROTON-GEM/.venv/lib64/python                
                             3.11/site-packages/unimol_tools/weights                                               

2026-02-02 02:38:57 | unimol_tools/weights/weighthub.py | 49 | INFO | Uni-Mol Tools | Downloading mol_pre_all_h_220816.pt


...

                    INFO     Number of GPUs available: 1                                              ]8;id=448377;file:///n/holylfs06/LABS/mzitnik_lab/Users/rzhu/PROTON-GEM/.venv/lib64/python3.11/site-packages/unimol_tools/tasks/trainer.py\trainer.py]8;;\:]8;id=91884;file:///n/holylfs06/LABS/mzitnik_lab/Users/rzhu/PROTON-GEM/.venv/lib64/python3.11/site-packages/unimol_tools/tasks/trainer.py#78\78]8;;\

2026-02-02 02:54:54 | unimol_tools/tasks/trainer.py | 98 | INFO | Uni-Mol Tools | Using single GPU.


                    INFO     Using single GPU.                                                        ]8;id=629371;file:///n/holylfs06/LABS/mzitnik_lab/Users/rzhu/PROTON-GEM/.venv/lib64/python3.11/site-packages/unimol_tools/tasks/trainer.py\trainer.py]8;;\:]8;id=503660;file:///n/holylfs06/LABS/mzitnik_lab/Users/rzhu/PROTON-GEM/.venv/lib64/python3.11/site-packages/unimol_tools/tasks/trainer.py#98\98]8;;\

100%|██████████████████| 1/1 [00:00<00:00, 43.86it/s]
Encoding molecules:  77%|▊| 167/217 [15:54<04:45,  5.


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:5                                                                                    │
│                                                                                                  │
│   2 smiles_list = valid_drugs_df["SMILES"].tolist()                                              │
│   3                                                                                              │
│   4 # Encode with Uni-Mol2                                                                       │
│ ❱ 5 unimol_embeddings = encode_molecules_unimol(smiles_list, cache_path=UNIMOL_CACHE_PATH)       │
│   6 _logger.info(f"Uni-Mol2 embeddings shape: {unimol_embeddings.shape}")                        │
│   7                                                                                              │
│                                                                                                  │
│ in encode_molecules_unimol:40                                                                    │
│                                                                                                  │
│   37 │   for i in tqdm(range(0, len(smiles_list), batch_size), desc="Encoding molecules"):       │
│   38 │   │   batch = smiles_list[i : i + batch_size]                                             │
│   39 │   │   # UniMolRepr returns dict with 'cls_repr' key containing [batch, 512] embeddings    │
│ ❱ 40 │   │   reprs = clf.get_repr(batch, return_atomic_reprs=False)                              │
│   41 │   │   # Extract CLS token representation (molecule-level)                                 │
│   42 │   │   batch_emb = torch.tensor(reprs["cls_repr"], dtype=torch.float32)                    │
│   43 │   │   all_embeddings.append(batch_emb)                                                    │
│                                                                                                  │
│ /n/holylfs06/LABS/mzitnik_lab/Users/rzhu/PROTON-GEM/.venv/lib64/python3.11/site-packages/unimol_ │
│ tools/predictor.py:129 in get_repr                                                               │
│                                                                                                  │
│   126 │   │   else:                                                                              │
│   127 │   │   │   raise ValueError('Unknown data type: {}'.format(type(data)))                   │
│   128 │   │                                                                                      │
│ ❱ 129 │   │   datahub = DataHub(                                                                 │
│   130 │   │   │   data=data,                                                                     │
│   131 │   │   │   task='repr',                                                                   │
│   132 │   │   │   is_train=False,                                                                │
│                                                                                                  │
│ /n/holylfs06/LABS/mzitnik_lab/Users/rzhu/PROTON-GEM/.venv/lib64/python3.11/site-packages/unimol_ │
│ tools/data/datahub.py:42 in __init__                                                             │
│                                                                                                  │
│    39 │   │   self.multiclass_cnt = params.get('multiclass_cnt', None)                           │
│    40 │   │   self.ss_method = params.get('target_normalize', 'none')                            │
│    41 │   │   self.conf_cache_level = params.get('conf_cache_level', 1)                          │
│ ❱  42 │   │   self._init_data(**params)                                                          │
│    43 │   │   self._init_split(**params)                                                         │
│    44 │                                                    

## Adapter Architecture and Training

Train a small MLP adapter to map Uni-Mol2 representations into PROTON's embedding space.

In [ ]:
class MolecularAdapter(nn.Module):
    """MLP adapter to map Uni-Mol2 molecular representations to PROTON embedding space.

    Architecture: input -> Linear -> LayerNorm -> GELU -> Dropout -> ... -> output
    """

    def __init__(
        self,
        input_dim: int = 512,
        hidden_dim: int = 512,
        output_dim: int = 512,
        num_layers: int = 2,
        dropout: float = 0.1,
    ):
        """Initialize the adapter.

        Args:
            input_dim: Uni-Mol2 embedding dimension
            hidden_dim: Hidden layer dimension
            output_dim: PROTON embedding dimension
            num_layers: Number of hidden layers
            dropout: Dropout probability
        """
        super().__init__()

        layers = []

        # Input layer
        layers.extend([
            nn.Linear(input_dim, hidden_dim),
            nn.LayerNorm(hidden_dim),
            nn.GELU(),
            nn.Dropout(dropout),
        ])

        # Hidden layers
        for _ in range(num_layers - 1):
            layers.extend([
                nn.Linear(hidden_dim, hidden_dim),
                nn.LayerNorm(hidden_dim),
                nn.GELU(),
                nn.Dropout(dropout),
            ])

        # Output layer
        layers.append(nn.Linear(hidden_dim, output_dim))

        self.layers = nn.Sequential(*layers)

    def forward(self, x: torch.Tensor) -> torch.Tensor:
        """Forward pass.

        Args:
            x: Uni-Mol2 embeddings [batch, input_dim]

        Returns:
            Projected embeddings [batch, output_dim]
        """
        return self.layers(x)

Prepare training data.

In [ ]:
# Split data into train/validation sets
X = unimol_embeddings.numpy()
y = valid_drug_embeddings.numpy()

X_train, X_val, y_train, y_val = train_test_split(
    X, y, test_size=0.2, random_state=conf.seed
)

_logger.info(f"Training samples: {len(X_train)}")
_logger.info(f"Validation samples: {len(X_val)}")

# Convert to tensors
X_train_t = torch.tensor(X_train, dtype=torch.float32)
y_train_t = torch.tensor(y_train, dtype=torch.float32)
X_val_t = torch.tensor(X_val, dtype=torch.float32)
y_val_t = torch.tensor(y_val, dtype=torch.float32)

# Create data loaders
train_dataset = TensorDataset(X_train_t, y_train_t)
val_dataset = TensorDataset(X_val_t, y_val_t)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=64, shuffle=False)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:2                                                                                    │
│                                                                                                  │
│    1 # Split data into train/validation sets                                                     │
│ ❱  2 X = unimol_embeddings.numpy()                                                               │
│    3 y = valid_drug_embeddings.numpy()                                                           │
│    4                                                                                             │
│    5 X_train, X_val, y_train, y_val = train_test_split(                                          │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
NameError: name 'unimol_embeddings' is not defined

Training loop with early stopping.

In [ ]:
def train_adapter(
    model: nn.Module,
    train_loader: DataLoader,
    val_loader: DataLoader,
    num_epochs: int = 500,
    lr: float = 1e-3,
    patience: int = 20,
    device: torch.device = TORCH_DEVICE,
    wandb_run: object | None = None,
 ) -> dict:
    """Train the adapter with early stopping.

    Args:
        model: MolecularAdapter model
        train_loader: Training data loader
        val_loader: Validation data loader
        num_epochs: Maximum number of epochs
        lr: Learning rate
        patience: Early stopping patience
        device: Torch device
        wandb_run: Optional wandb run for metric logging

    Returns:
        Training history dict
    """

    model = model.to(device)
    optimizer = torch.optim.AdamW(model.parameters(), lr=lr, weight_decay=1e-4)
    scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
        optimizer, mode="min", factor=0.5, patience=10
    )
    criterion = nn.MSELoss()

    history = {"train_loss": [], "val_loss": [], "cosine_sim": []}
    best_val_loss = float("inf")
    best_state = None
    patience_counter = 0

    pbar = tqdm(range(num_epochs), desc="Training adapter")
    for epoch in pbar:
        # Training
        model.train()
        train_loss = 0.0
        for X_batch, y_batch in train_loader:
            X_batch = X_batch.to(device)
            y_batch = y_batch.to(device)

            optimizer.zero_grad()
            y_pred = model(X_batch)
            loss = criterion(y_pred, y_batch)
            loss.backward()
            optimizer.step()

            train_loss += loss.item() * len(X_batch)

        train_loss /= len(train_loader.dataset)

        # Validation
        model.eval()
        val_loss = 0.0
        all_preds = []
        all_targets = []

        with torch.no_grad():
            for X_batch, y_batch in val_loader:
                X_batch = X_batch.to(device)
                y_batch = y_batch.to(device)

                y_pred = model(X_batch)
                loss = criterion(y_pred, y_batch)
                val_loss += loss.item() * len(X_batch)

                all_preds.append(y_pred.cpu())
                all_targets.append(y_batch.cpu())

        val_loss /= len(val_loader.dataset)

        # Compute cosine similarity
        all_preds = torch.cat(all_preds, dim=0)
        all_targets = torch.cat(all_targets, dim=0)
        cosine_sim = F.cosine_similarity(all_preds, all_targets, dim=1).mean().item()

        # Update history
        history["train_loss"].append(train_loss)
        history["val_loss"].append(val_loss)
        history["cosine_sim"].append(cosine_sim)

        # Learning rate scheduling
        scheduler.step(val_loss)

        current_lr = optimizer.param_groups[0]["lr"]
        if wandb_run is not None:
            wandb_run.log(
                {
                    "adapter/epoch": epoch + 1,
                    "adapter/train_loss": train_loss,
                    "adapter/val_loss": val_loss,
                    "adapter/val_cosine_sim": cosine_sim,
                    "adapter/lr": current_lr,
                },
                step=epoch + 1,
            )

        # Update progress bar
        pbar.set_postfix({
            "train": f"{train_loss:.4f}",
            "val": f"{val_loss:.4f}",
            "cos": f"{cosine_sim:.4f}",
        })

        # Early stopping
        if val_loss < best_val_loss:
            best_val_loss = val_loss
            best_state = copy.deepcopy(model.state_dict())
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= patience:
                _logger.info(f"Early stopping at epoch {epoch + 1}")
                break

    # Restore best model
    if best_state is not None:
        model.load_state_dict(best_state)

    if wandb_run is not None:
        wandb_run.summary["adapter/best_val_loss"] = best_val_loss

    return history

Train the adapter.

In [ ]:
# Initialize adapter
adapter = MolecularAdapter(
    input_dim=unimol_embeddings.shape[1],
    hidden_dim=512,
    output_dim=EMBEDDING_DIM,
    num_layers=2,
    dropout=0.1,
 )

_logger.info(f"Adapter parameters: {sum(p.numel() for p in adapter.parameters()):,}")

# W&B run (uses WANDB_API_KEY from env or secrets.env)
wandb_run = None
try:
    wandb_run = wandb.init(
        entity="rzhu52-harvard-university",
        project="PROTON_GEM",
        name="molecular_adapter"
    )
    wandb.watch(adapter, log="gradients", log_freq=200)
except Exception as e:
    _logger.warning(f"W&B init failed; continuing without wandb logging. Error: {e}")
    wandb_run = None

# Train
history = train_adapter(
    adapter,
    train_loader,
    val_loader,
    num_epochs=500,
    lr=1e-3,
    patience=20,
    wandb_run=wandb_run,
 )

if wandb_run is not None:
    wandb_run.finish()

Save trained adapter.

In [ ]:
# Save adapter
torch.save({
    "model_state_dict": adapter.state_dict(),
    "input_dim": unimol_embeddings.shape[1],
    "hidden_dim": 512,
    "output_dim": EMBEDDING_DIM,
    "num_layers": 2,
}, ADAPTER_PATH)
_logger.info(f"Saved adapter to {ADAPTER_PATH}")

Plot training curves.

In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# Loss curves
ax = axes[0]
ax.plot(history["train_loss"], label="Train", color="#1f77b4")
ax.plot(history["val_loss"], label="Validation", color="#ff7f0e")
ax.set_xlabel("Epoch")
ax.set_ylabel("MSE Loss")
ax.set_title("Training Loss")
ax.legend()
ax.grid(True, alpha=0.3)

# Cosine similarity
ax = axes[1]
ax.plot(history["cosine_sim"], color="#2ca02c")
ax.set_xlabel("Epoch")
ax.set_ylabel("Cosine Similarity")
ax.set_title("Validation Cosine Similarity")
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(OUTPUT_DIR / "training_curves.pdf", bbox_inches="tight")
plt.show()

## Evaluation

Evaluate the adapter's ability to reconstruct PROTON embeddings from molecular structure.

In [ ]:
# Compute adapted embeddings for all valid drugs
adapter.eval()
adapter = adapter.to(TORCH_DEVICE)

with torch.no_grad():
    adapted_embeddings = adapter(unimol_embeddings.to(TORCH_DEVICE)).cpu()

_logger.info(f"Adapted embeddings shape: {adapted_embeddings.shape}")

Compute per-drug cosine similarity.

In [ ]:
# Compute cosine similarity for each drug
cosine_similarities = F.cosine_similarity(
    adapted_embeddings, valid_drug_embeddings, dim=1
).numpy()

# Add to dataframe
valid_drugs_df["cosine_similarity"] = cosine_similarities

# Statistics
_logger.info(f"Mean cosine similarity: {cosine_similarities.mean():.4f}")
_logger.info(f"Std cosine similarity: {cosine_similarities.std():.4f}")
_logger.info(f"Min cosine similarity: {cosine_similarities.min():.4f}")
_logger.info(f"Max cosine similarity: {cosine_similarities.max():.4f}")

Plot cosine similarity distribution.

In [ ]:
fig, ax = plt.subplots(figsize=(8, 5))

ax.hist(cosine_similarities, bins=50, color="#1f77b4", edgecolor="black", alpha=0.7)
ax.axvline(cosine_similarities.mean(), color="red", linestyle="--", 
           label=f"Mean: {cosine_similarities.mean():.3f}")
ax.set_xlabel("Cosine Similarity")
ax.set_ylabel("Count")
ax.set_title("Distribution of Adapted vs. Original Embedding Similarity")
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(OUTPUT_DIR / "cosine_similarity_distribution.pdf", bbox_inches="tight")
plt.show()

Identify best and worst reconstructed drugs.

In [ ]:
# Best reconstructed drugs
_logger.info("Top 10 best reconstructed drugs:")
best_drugs = valid_drugs_df.nlargest(10, "cosine_similarity")[["node_name", "cosine_similarity"]]
display(best_drugs)

# Worst reconstructed drugs
_logger.info("Top 10 worst reconstructed drugs:")
worst_drugs = valid_drugs_df.nsmallest(10, "cosine_similarity")[["node_name", "cosine_similarity"]]
display(worst_drugs)

## De Novo Molecule Inference

Use the trained adapter to project new molecules into PROTON's embedding space and score them against diseases.

In [ ]:
def load_adapter(checkpoint_path: Path, device: torch.device = TORCH_DEVICE) -> MolecularAdapter:
    """Load a trained adapter from checkpoint.

    Args:
        checkpoint_path: Path to adapter checkpoint
        device: Torch device

    Returns:
        Loaded MolecularAdapter model
    """
    checkpoint = torch.load(checkpoint_path, map_location=device)
    adapter = MolecularAdapter(
        input_dim=checkpoint["input_dim"],
        hidden_dim=checkpoint["hidden_dim"],
        output_dim=checkpoint["output_dim"],
        num_layers=checkpoint["num_layers"],
    )
    adapter.load_state_dict(checkpoint["model_state_dict"])
    adapter = adapter.to(device)
    adapter.eval()
    return adapter


def project_denovo_molecule(
    smiles: str,
    adapter: MolecularAdapter,
    device: torch.device = TORCH_DEVICE,
) -> torch.Tensor:
    """Project a de novo molecule into PROTON's embedding space.

    Args:
        smiles: SMILES string of the molecule
        adapter: Trained MolecularAdapter
        device: Torch device

    Returns:
        Embedding tensor [1, 512]
        
    Raises:
        ValueError: If SMILES encoding fails
    """
    # Import Uni-Mol2
    from unimol_tools import UniMolRepr

    # Encode with Uni-Mol2
    try:
        clf = UniMolRepr(data_type="molecule", remove_hs=False)
        reprs = clf.get_repr([smiles], return_atomic_reprs=False)
        mol_embedding = torch.tensor(reprs["cls_repr"], dtype=torch.float32).to(device)
    except Exception as e:
        raise ValueError(f"Failed to encode molecule: {smiles}. Error: {e}")

    # Project through adapter
    with torch.no_grad():
        projected = adapter(mol_embedding)

    return projected


def score_molecule_disease(
    smiles: str,
    disease_index: int,
    adapter: MolecularAdapter,
    model: HGT,
    kg: object,
    embeddings: torch.Tensor,
    edge_type: tuple = ("disease", "indication", "drug"),
) -> float:
    """Score a de novo molecule against a disease.

    Args:
        smiles: SMILES string of the molecule
        disease_index: Node index of the disease
        adapter: Trained MolecularAdapter
        model: PROTON HGT model
        kg: Knowledge graph
        embeddings: Pre-computed PROTON embeddings
        edge_type: Edge type tuple for scoring

    Returns:
        Indication score (higher = more likely)
    """
    # Get molecule embedding
    mol_embedding = project_denovo_molecule(smiles, adapter)

    # Create a modified embeddings tensor with the new molecule
    # We'll use a placeholder index (the first drug index)
    drug_nodes = nodes[nodes["node_type"] == "drug"]
    placeholder_idx = drug_nodes["node_index"].iloc[0]

    # Clone embeddings and replace placeholder
    modified_embeddings = embeddings.clone()
    modified_embeddings[placeholder_idx] = mol_embedding.squeeze().cpu()

    # Get decoder weights
    decoder = model.decoder.W.detach().cpu()

    # Score
    scores = model.get_scores_from_embeddings(
        src_ids=[disease_index],
        dst_ids=[placeholder_idx],
        query_edge_type=edge_type,
        embeddings=modified_embeddings,
        decoder=decoder,
        query_kg=kg,
        use_cache=False,
    )

    return scores[0].item()

### Example: Score Novel Molecules Against Parkinson's Disease

In [ ]:
# Find Parkinson's disease index
pd_node = nodes[nodes["node_name"] == "Parkinson disease"]
pd_index = pd_node["node_index"].values[0]
_logger.info(f"Parkinson's disease index: {pd_index}")

In [ ]:
# Example: Score Levodopa (known PD drug) as a sanity check
# Levodopa SMILES: NC(Cc1ccc(O)c(O)c1)C(=O)O
levodopa_smiles = "NC(Cc1ccc(O)c(O)c1)C(=O)O"

# Score against Parkinson's disease
score = score_molecule_disease(
    smiles=levodopa_smiles,
    disease_index=pd_index,
    adapter=adapter,
    model=pretrain_model,
    kg=kg,
    embeddings=embeddings,
)

_logger.info(f"Levodopa indication score for Parkinson's disease: {score:.4f}")

Compare with existing drug scores for reference.

In [ ]:
# Get decoder
decoder = pretrain_model.decoder.W.detach().cpu()

# Score all existing drugs against Parkinson's disease
all_drug_indices = drug_nodes["node_index"].values
pd_indices = [pd_index] * len(all_drug_indices)

all_scores = pretrain_model.get_scores_from_embeddings(
    src_ids=pd_indices,
    dst_ids=all_drug_indices.tolist(),
    query_edge_type=("disease", "indication", "drug"),
    embeddings=embeddings,
    decoder=decoder,
    query_kg=kg,
    use_cache=False,
)

# Statistics
all_scores_np = all_scores.cpu().numpy()
_logger.info(f"Existing drug scores - Mean: {all_scores_np.mean():.4f}, Std: {all_scores_np.std():.4f}")
_logger.info(f"Existing drug scores - Min: {all_scores_np.min():.4f}, Max: {all_scores_np.max():.4f}")

### Batch Scoring for Multiple Molecules

In [ ]:
def score_molecules_batch(
    smiles_list: list[str],
    disease_index: int,
    adapter: MolecularAdapter,
    model: HGT,
    kg: object,
    embeddings: torch.Tensor,
    device: torch.device = TORCH_DEVICE,
) -> pd.DataFrame:
    """Score multiple molecules against a disease.
    
    Automatically skips invalid molecules that cannot be encoded.

    Args:
        smiles_list: List of SMILES strings
        disease_index: Node index of the disease
        adapter: Trained MolecularAdapter
        model: PROTON HGT model
        kg: Knowledge graph
        embeddings: Pre-computed PROTON embeddings
        device: Torch device

    Returns:
        DataFrame with valid SMILES and scores
    """
    # Use robust encoding function (reuses the logic defined earlier)
    # This handles invalid SMILES gracefully by skipping them
    try:
        mol_embeddings, valid_indices = encode_molecules_unimol(
            smiles_list, cache_path=None, batch_size=32
        )
    except ValueError:
        _logger.warning("All molecules in batch failed to encode.")
        return pd.DataFrame(columns=["SMILES", "score"])

    mol_embeddings = mol_embeddings.to(device)
    
    # Filter smile list to valid ones
    valid_smiles = [smiles_list[i] for i in valid_indices]
    
    # Report if some were dropped
    if len(valid_indices) < len(smiles_list):
        _logger.warning(f"Dropped {len(smiles_list) - len(valid_indices)} invalid molecules from batch.")

    # Project through adapter
    with torch.no_grad():
        projected = adapter(mol_embeddings).cpu()

    # Score each molecule
    scores = []
    drug_nodes_local = nodes[nodes["node_type"] == "drug"]
    placeholder_idx = drug_nodes_local["node_index"].iloc[0]
    decoder = model.decoder.W.detach().cpu()

    for i, emb in enumerate(projected):
        modified_embeddings = embeddings.clone()
        modified_embeddings[placeholder_idx] = emb

        score = model.get_scores_from_embeddings(
            src_ids=[disease_index],
            dst_ids=[placeholder_idx],
            query_edge_type=("disease", "indication", "drug"),
            embeddings=modified_embeddings,
            decoder=decoder,
            query_kg=kg,
            use_cache=False,
        )
        scores.append(score[0].item())

    return pd.DataFrame({"SMILES": valid_smiles, "score": scores})

Example: Score a set of candidate molecules.

In [ ]:
# Example candidate molecules (known PD drugs and random molecules)
candidate_smiles = [
    "NC(Cc1ccc(O)c(O)c1)C(=O)O",  # Levodopa
    "CC(C)CC1=CC=C(C=C1)C(C)C(=O)O",  # Ibuprofen (not PD drug)
    "CN1C=NC2=C1C(=O)N(C(=O)N2C)C",  # Caffeine
    "CC(=O)OC1=CC=CC=C1C(=O)O",  # Aspirin
]

candidate_names = ["Levodopa", "Ibuprofen", "Caffeine", "Aspirin"]

# Score candidates
results = score_molecules_batch(
    smiles_list=candidate_smiles,
    disease_index=pd_index,
    adapter=adapter,
    model=pretrain_model,
    kg=kg,
    embeddings=embeddings,
)

results["name"] = candidate_names
results = results[["name", "SMILES", "score"]].sort_values("score", ascending=False)

_logger.info("Candidate molecule scores for Parkinson's disease:")
display(results)

## Summary

This notebook demonstrated how to:

1. **Collect SMILES data** for existing drugs in NeuroKG from PubChem
2. **Encode molecules** using Uni-Mol2's pre-trained molecular encoder
3. **Train an adapter** to map Uni-Mol2 representations to PROTON's embedding space
4. **Project de novo molecules** into PROTON's space and score them against diseases

### Key Outputs

| File | Description |
|------|-------------|
| `data/neurokg/drugs/drug_smiles.csv` | Cached SMILES for NeuroKG drugs |
| `data/neurokg/drugs/unimol_embeddings.pt` | Cached Uni-Mol2 embeddings |
| `data/checkpoints/molecular_adapter.pt` | Trained adapter weights |

### Usage

To score a new molecule:

```python
# Load adapter
adapter = load_adapter(ADAPTER_PATH)

# Score molecule against disease
score = score_molecule_disease(
    smiles="YOUR_SMILES_HERE",
    disease_index=disease_index,
    adapter=adapter,
    model=pretrain_model,
    kg=kg,
    embeddings=embeddings,
)
```